# TL;DR
本文主要介绍：
1. 通过近似二项分布构建了token向量在*LLM*的*MLP层*经过`ReLU`激活函数前为何升维、升多少维的的有效特征概率模型。
  - 参考：[OpenAI开源模型gpt-oss-120b的妙妙小观察](https://zhuanlan.zhihu.com/p/1934722616544954132)
2. Llama模型*MLP使用的是`SwiGLU`激活函数，在该激活函数下，升维有何说法。(to be continued...)
  - 参考：[OpenAI开源模型gpt-oss-120b的妙妙小观察](https://zhuanlan.zhihu.com/p/1934722616544954132)
  - 参考：[The Case for Co-Designing Model Architectures with Hardware](https://arxiv.org/pdf/2401.14489)
  - 参考：[Why isn't intermediate_size 4 * hidden_size for Llama as in paper?](https://github.com/huggingface/transformers/issues/27139)

# 1. 通过近似二项分布构建了token向量在*LLM*的*MLP层*经过`ReLU`激活函数前为何升维、升多少维的的有效特征概率模型。
> Why does intermediate size is hidden_size x 4 in LLM?

## Premises and Assumptions
- 使用激活函数ReLU（若是其他激活函数命题可能不成立）
- 单token向量维度为`n`，即`hidden_size`
- 线性层升维后的维度`intermediate size`设为`m`
- ReLU函数表达式：$ReLU(x)=max(0,x)=\frac{x+|x|}{2}$（为啥第二个公式？反向传播ReLU求导用的就是这个。。。）
<p>ReLU函数图像</p>
<p><img alt="ReLU" src="https://docs.pytorch.org/docs/stable/_images/ReLU.png"  width="40%" height="40%" align="center"/></p>

### 明确研究模型与命题
- 研究模型：$output=ReLU(Linear(x))$。其中`x`即单token向量，`Linear`即升维线性层，`ReLU`即激活函数
- 命题：一行token向量在经过激活函数后的值若含0维度>原token向量维度，我们会认为这个token向量的有效特征的数量变少，即可表达的信息变少。因此需要通过一层线性层升维（即`Linear`层）后再经过`ReLU`激活，减少特征损失。但是通过`Linear`层升维到多少才不会导致激活后特征减少？就是我们需要研究的命题。即：
$$
output=ReLU(Linear(x)),x \in R^{1*n},Linear \in R^{n*m},output \in R^{1*m}\\
m=?使得(output==0).sum()>=(x==0).sum()
$$

## 推论步骤
我们可以先定义随机变量：<br>
设$K$是单token向量激活后非零维度的数量（即有效特征的数量），$K$是一个随机变量且有$K \in [0,m]$

概率假设：<br>
因为对于单个维度，经过$ReLU$后要么为零（未激活）要么非零（被激活），因此假设对各个维度的激活概率为$0.5$

假设：<br>
由于各个维度间独立，因此我们可以假设对**各个维度的激活设为独立的伯努利试验**，则对整个token向量的激活，我们可以假设其服从二项分布。<br>
即有：$K \sim Binomial(m,p=0.5)$<br>
则$K=k\text{（k为激活维度数）}$的概率我们可以用二项分布的概率质量函数表示如下：
$$P(K=k)=C^k_m*(0.5)^k*(1-0.5)^{m-k}=C^k_m*(0.5)^m$$

而我们需要计算的概率是$P(K>=n)$，即非零维度数至少为$n$，保证表达能力不降低。则有：<br>
$$P(K>=n)=P(K=n)+P(K=n+1)+P(K=n+2)+\dots+P(K=m)=\sum^{m}_{k=n}P(K=k)$$

代入概率质量函数得：
$$P(K>=n)=\sum^{m}_{k=n}P(K=k)=\sum^{m}_{k=n}C^k_m*(0.5)^m=\frac{\sum^{m}_{k=n}C^k_m}{2^m}$$

得出结论：m的值应尽可能使$P(K>=n)$接近于100%，使得激活后有效特征数量基本不会有下降的可能。因此有：
$$f(m)=P(K>=n)，需要求argmax(f(m))$$

In [ ]:
# 链接：https://www.zhihu.com/question/665731716/answer/1888209852712600269
import math

def C(x,y):
    """
    排列C^x_y = y!/x!*(y-x)!
    """
    return math.factorial(y)/(math.factorial(x)*math.factorial(y-x))

def rank_dec_ratio(m,n):
    """
    Args:
        m: intermediate size
        n: hidden size
    """
    sum = 0
    for k in range(n,m,1):
        sum += C(k,m)
    return 1- sum/(2**m) #XXX 1-P(K>=n)，求的是argmin(1-f(m))，越接近0越满足结论

for n in range(1,10,1):
    #XXX n：hidden_size，这里只测1~10
    print(f"hidden_size={n}  ",rank_dec_ratio(2*n,n),rank_dec_ratio(3*n,n),rank_dec_ratio(4*n,n))

hidden_size=1   0.5 0.25 0.125
hidden_size=2   0.375 0.125 0.0390625
hidden_size=3   0.359375 0.091796875 0.01953125
hidden_size=4   0.3671875 0.0732421875 0.010650634765625
hidden_size=5   0.3779296875 0.05926513671875 0.005909919738769531
hidden_size=6   0.387451171875 0.048130035400390625 0.0033054351806640625
hidden_size=7   0.39532470703125 0.03917741775512695 0.0018595866858959198
hidden_size=8   0.401824951171875 0.031957387924194336 0.001051201019436121
hidden_size=9   0.4072685241699219 0.026119500398635864 0.0005966214957879856


从上面的代码我们可以得出结论：
1. `hidden size`相同时，`intermediate size`为`hidden size`*2时，激活后有很大部分维度为0。因此升维不可能只x2
2. 升维维度>2且相同时，`hidden size`越大，激活后有维度基本不会为0，可以保留基本全部的有效特征。

然后现在业界一般取x4，主要是为了凑`intermediate size`为2的指数，这样在做分布式训练，infra时能工整地切片

证毕

In [1]:
import math

def compute_intermediate_size(n):
    return int(math.ceil(n * 8 / 3) + 255) // 256 * 256

compute_intermediate_size(4096)

11008